# HW 2 - Analysis 2 - Batch by File Name

***I'm sure it can be cleaner, and made into some kind of function doc like the whatif.py?*** 

***It's not exactly what you asked for, but it will batch every three letter combination the user initiates***

## Please use FCM for the example

***I deleted the directories and files, so you can see what it does - otherwise there are errors because the folders already exist, duplicate headers, etc.***

## Preliminaries

In [ ]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

Import commonly used libraries and magic command for inline plotting

In [ ]:
import numpy as np
import pandas as pd
from pandas import read_csv
import openpyxl
from openpyxl.styles import Alignment
import csv
import shutil
import glob
import os
from pathlib import Path

## Step 1 – Consolidation

***Ask for which file types to batch - use FCM***

In [ ]:
print('Enter the three letter file ID to batch:')
f_id = input()
print('We will begin batch processing log files that begin with: ' + f_id)

***Make the new directory and copy the original files first, so only modifying copies***

In [ ]:
# Copy all original files

# Specify the source directory path
source_directory = './data/logs/'

copies_directory = './data/log_copies/'

destination_directory = os.path.join(copies_directory, f_id)

os.mkdir(destination_directory)

# Get a list of all files in the source directory
files = glob.glob(source_directory + '/' + f_id + '*.csv')

# Copy each file to the destination directory
for file in files:
   shutil.copy(file, destination_directory)

***I wanted to confirm the destination_directory could be used in the remainder of the processes***

In [ ]:
destination_directory

***Added a header to each csv file in the copies folder***

In [ ]:
# Add header to each file copy

filelist = glob.glob(destination_directory + "/*.csv")
df_list = []
for file in filelist:
# you also dont need to add path, the glob should already have the full path
    df2 = read_csv(file,names=['datetime','scale','temperature'])
    ## save out files
    df2.to_csv(file,index=False)
    df_list.append(df2)
frame = pd.concat(df_list)

***Places each csv file onto one sheet in the workbook, this also creates the file - I'm going to use different files names as I step through this process***

In [ ]:
# Copy each csv file to a workbook sheet

path = destination_directory

def write_sheets(file_map: dict) -> None:
    with pd.ExcelWriter(f"{path}/{f_id}_v1.xlsx", engine="xlsxwriter") as writer:
        [df.to_excel(writer, sheet_name=sheet_name, index=False) for sheet_name, df in file_map.items()]

file_mapping = {Path(file).stem: pd.read_csv(file) for file in Path(path).glob("*.csv")}
write_sheets(file_mapping)

### Step 2 - Summarization

***Sorting each sheet by column A, to get the min and max datetime values later***

In [ ]:
# Sort each sheet by column A, in version 1

# Load the Excel workbook
workbook = openpyxl.load_workbook(f"{destination_directory}/{f_id}_v1.xlsx")

# Iterate through each sheet
for sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]
    
    # Extract data from column A
    column_a_data = []
    for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=1):
        column_a_data.append(row[0].value)
    
    # Sort rows based on values in column A
    sorted_rows = sorted(sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1),
                          key=lambda x: x[0].value)
    
    # Rewrite sorted data back to the sheet
    for idx, row in enumerate(sorted_rows, start=2):
        for col_idx, cell in enumerate(row, start=1):
            sheet.cell(row=idx, column=col_idx).value = cell.value

# Save the changes to the workbook
workbook.save(f"{destination_directory}/{f_id}_sorted.xlsx")

***Adding the formulas to the sorted file, adjusting the columns to autofit the width, and fixing the alignment of the min/max datetimes***

In [ ]:
# Add the formulas to each workbook sheet - using the sorted file

# Load the Excel workbook
workbook = openpyxl.load_workbook(f"{destination_directory}/{f_id}_sorted.xlsx")

# Iterate through each sheet in the workbook
for sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]
    
    # Add labels to cells G2:G7
    sheet['G2'] = 'min_temp'
    sheet['G3'] = 'max_temp'
    sheet['G4'] = 'mean_temp'
    sheet['G6'] = 'min_date'
    sheet['G7'] = 'max_date'
    
    # Add formulas to cell H2:H7
    sheet['H2'] = '=ROUND(MIN(C:C),1)'
    sheet['H3'] = '=ROUND(MAX(C:C),1)'
    sheet['H4'] = '=ROUND(AVERAGE(C:C),1)'
    sheet['H6'] = '=A2'
    col_a_length = len(sheet['A'])
    sheet['H7'] = '=A' + str(col_a_length)
    
    # Align cells H6 and H7 to the right
    sheet['H6'].alignment = Alignment(horizontal='right')
    sheet['H7'].alignment = Alignment(horizontal='right')
    
    # Iterate through each column in the worksheet
    for col in sheet.columns:
        max_length = 0
        
        # Iterate through each cell in the column to find the maximum length
        for cell in col:
            try:
                # Check if the cell contains a string and update the maximum length
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        
        # Autofit the column width based on the maximum length
        sheet.column_dimensions[col[0].column_letter].width = max_length + 2  # Add some extra space

# Save the changes to the workbook
workbook.save(f"{destination_directory}/{f_id}_Final.xlsx")

***The final .xlsx should be all set***